# Module 10 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [1]:
from unification import parse, unification

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

-----

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

<a id="find_successors"></a>
## find_successors

Find the successors for a given 'goal' state. Could be a list of none to many.

Variables
* **next** list: goal state
* **states** list[list]: list of states
* **actions** dictionary: dictionary of permissible actions

**returns** list[list]: list of successor states

In [2]:
def find_successors(next, states, actions):
    next_state_id = next[0]
    successors = []
    if next_state_id in actions:
        for condition in actions[next_state_id]["conditions"]:
            if not condition in states:
                successors.append(condition)
    #if not next in states:
        # need to get this state out of here somehow..
        #successors.append()
    return successors

<a id="parse_states"></a>
## parse_states

Parse states from a list into a list of lists so it is more usable.

Variables
* **states** List: states

**returns** List[List]: list of lists which are states

In [3]:
def parse_states(states):
    return [parse(x) for x in states]

<a id="parse_actions"></a>
## parse_actions

Parse actions from a dictionary to be lists so it is more usable.

Variables
* **actions** Dictionary: dictionary of actions that have sublists of states

**returns** Dictionary: same dictionary items but instead of () states and actions they are now all lists so it's more useable

In [4]:
def parse_actions(actions):
    keys = ["drive", "buy"]
    for key in keys:
        actions[key]["action"] = parse(actions[key]["action"])
        actions[key]["conditions"] = [parse(x) for x in actions[key]["conditions"]]
        actions[key]["add"] = [parse(x) for x in actions[key]["add"]]
        actions[key]["delete"] = [parse(x) for x in actions[key]["delete"]]
    return actions

<a id="forward_planner"></a>
## forward_planner

Plan forward. Takes the start state, goal state, and all permissible actions, then finds the plan to go from the starting state to the goal state.

Variables
* **start_state** List: states
* **goal** List: states
* **actions** dictionary: actions that can be taken, contains sub dictionaries of states
* **debug** bool: default false, when true we print out states at every while loop

**returns** List: list of all planned actions to take to get from starting to goal state

In [20]:
def forward_planner(start_state, goal, actions, debug=False):
    parsed_state = parse_states(start_state)
    parsed_goal = parse_states(goal)
    parsed_actions = parse_actions(actions)
    plan = []
    while (set(map(tuple, parsed_state)) != set(map(tuple, parsed_goal))):
        if debug:
            print("current state")
            print(parsed_state)
            print("current plan")
            print(plan)
        print(parsed_goal)
        successors = find_successors(parsed_goal[-1], parsed_state, parsed_actions)
        if len(successors) > 0:
            for s in successors:
                parsed_goal.append(s)
                # these could also be things we need to do so it could go into our plan maybe?
            # still need some kind of mechanism to unwind states incase we need to:
                # go to the store to get an item or we are at the store and want to be home, etc.
        else:
            # then we can assume 'next' has all of its conditions in the state already so we're good to move on
            parsed_goal.pop()
    return plan

You will be solving the problem from above. Here is the start state:

In [21]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [22]:
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"
]

and the actions/operators:

In [23]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

**Note** The facts for each state are really an ordered set. When comparing two states, you may need to convert them to a Set first.

In [24]:
plan = forward_planner(start_state, goal, actions)

[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['place', 'Bank'], ['agent', 'Me'], ['at', 'Me', 'Home'], ['at', 'Drill', 'Me'], ['at', 'Saw', 'Store']]
[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['place', 'Bank'], ['agent', 'Me'], ['at', 'Me', 'Home'], ['at', 'Drill', 'Me']]
[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['place', 'Bank'], ['agent', 'Me'], ['at', 'Me', 'Home']]
[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['place', 'Bank'], ['agent', 'Me']]
[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['place', 'Bank']]
[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home'], ['place', 'Store']]
[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home']]
[['item', 'Saw'], ['item', 'Drill']]
[['item', 'Saw']]
[]


IndexError: list index out of range

In [ ]:
for el in plan:
    print(el)

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.

I hope it wasn't expected that we would actually fully finish this. To get as far as I did I spent about 5 hours coding and trying to understand the lectures and decipher how we were expected to do this here but not use the algorithms we had just learned and used for the self-check.

Overall this assignment felt pretty unreasonable to me. We learning the algos in the lectures then applied them in the self check, why are we not supposed to use them here?? For doing this without the algos, I struggled to translate the DFS approach to this with finding the frontier and navigating it for the different possible successors, I really needed more hand-holding than what was given. I think this week would have been so much more beneficial to my learning if we got a partial implementation of STRIPS or the graph planning been provided and we have to finish it or something like that. Really displeased with the assignment this week, it felt like a waste.